In [2]:
import cv2

# Load the pre-trained Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open the webcam (use 0 for the default camera)
cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the webcam
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Draw rectangles around detected faces and label them as "Unknown"
    for (x, y, w, h) in faces:
        # Draw rectangle around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        # Label the face as "Unknown"
        cv2.putText(frame, 'Unknown', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Display the frame
    cv2.imshow('Face Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()


In [1]:
import cv2
import mediapipe as mp
from PIL import Image, ImageDraw, ImageFont
import numpy as np

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()

# Set the font path for modern text rendering (update this for your OS)
font_path = "Roboto-Light.ttf"  # Update with your font path
font = ImageFont.truetype(font_path, 24)  # Font size 24 for modern look

# Load a small background image for the text
text_background_path = "background.png"  # Path to your text background image
text_background = Image.open(text_background_path)

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to RGB for MediaPipe processing
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame for facial landmarks
    result = face_mesh.process(rgb_frame)

    # Convert the frame to a Pillow image for rendering modern text
    pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(pil_image)

    # Draw landmarks and names
    if result.multi_face_landmarks:
        for face_landmarks in result.multi_face_landmarks:
            # Get approximate top of the face for text placement
            top_x = int(face_landmarks.landmark[10].x * frame.shape[1])  # Landmark near forehead
            top_y = int(face_landmarks.landmark[10].y * frame.shape[0]) - 30  # Place text above forehead

            # Draw landmarks using OpenCV (retain blue points)
            for landmark in face_landmarks.landmark:
                x = int(landmark.x * frame.shape[1])
                y = int(landmark.y * frame.shape[0])
                cv2.circle(frame, (x, y), 1, (255, 0, 0), -1)  # Blue points

            # Resize and paste the text background image
            bg_width, bg_height = 300, 300  # Adjust size for your text background
            resized_bg = text_background.resize((bg_width, bg_height))
            pil_image.paste(resized_bg, (top_x - 150, top_y - 20), resized_bg)  # Transparent paste (RGBA)

            # Add name and subtitle on top of the background
            draw.text((top_x - 150, top_y + 290), "MOHAMMED S. ALZIYAD", font=font, fill=(255, 255, 255))  # Name in blue
            draw.text((top_x - 150, top_y + 310), "Your best friend", font=font, fill=(255, 255, 255))  # Subtitle

    # Convert the updated Pillow image back to OpenCV format
    frame_with_text_bg = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)

    # Display the frame
    cv2.imshow("Face Mesh with Text Background", frame_with_text_bg)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1732806139.926541 40795586 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1732806139.945866 40796110 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1732806139.952176 40796110 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1732806141.264955 40796108 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
